In [ ]:
import sagemaker
import boto3
import os
import shutil

In [ ]:
SESSION = sagemaker.Session()
BUCKET_NAME = SESSION.default_bucket()
FOLDER = 'capstone-project'
S3 = boto3.client('s3')
BUCKET = boto3.Session().resource('s3').Bucket(BUCKET_NAME)
DATA_DIRECTORY = 'data'
MODEL_DIRECTORY = 'model'

In [ ]:
def save_data_to_s3(data_directory):
    for entry in os.scandir(data_directory):
        local_file =  os.path.join(data_directory, entry.name)
        s3_file = os.path.join(FOLDER, local_file)
        BUCKET.Object(s3_file).upload_file(local_file)
        os.remove(local_file)

In [ ]:
def save_models_to_s3(model_directory):
    for entry in os.scandir(model_directory):
        if(entry.is_dir()):
            print(f'Zipping {entry.path} ...')
            shutil.make_archive(entry.path, 'zip', model_directory)
            zip_file = f'{os.path.join(model_directory, entry.name)}.zip'
            s3_file = os.path.join(FOLDER, zip_file)
            print(f'Uploading {zip_file} to {s3_file} ...')
            BUCKET.Object(s3_file).upload_file(zip_file)
            print(f'Removing {zip_file} ...')
            os.remove(zip_file)
            print(f'Removing {entry.path} ...')
            shutil.rmtree(entry.path)

In [ ]:
def get_data_from_s3(data_directory, filenames):
    for filename in filenames:
        s3_file = os.path.join(FOLDER, data_directory, filename)
        local_file = os.path.join(data_directory, filename)
        print(f'Downloading {s3_file} to {local_file}...')
        S3.download_file(BUCKET_NAME, s3_file, local_file)

In [ ]:
get_data_from_s3(DATA_DIRECTORY, ['data.csv', 'recipes.json'])

In [ ]:
save_models_to_s3(MODEL_DIRECTORY)